### Imports ->

In [272]:
import pandas as pd
import os
import plotly.express as px

### Obtendo código do município de análise ->

In [290]:
# Definição do município da análise (DIGITAR O NOME DO MUNICÍPIO)
municipio = 'Caraguatatuba'
estado = 'São Paulo'

# Carregando lista de códigos de municípios do IBGE
df_codigos_municipios = pd.read_excel('RELATORIO_DTB_BRASIL_MUNICIPIO.xls')

# Atribuição do código do município a uma variável
codigo_municipio = int(str(list(df_codigos_municipios.loc[df_codigos_municipios['Nome_Município'] == municipio, 'Código Município Completo'])[0])[0:6])

In [274]:
# Lista os arquivos das bases de dados mensais
lista_arquivos = os.listdir() # Lista dos arquivos da pasta do projeto
lista_arquivos = [arquivo for arquivo in lista_arquivos if ".txt" in arquivo] # Lista apenas dos arquivos .txt

In [275]:
# Cria dicionário de data frames
dic_df = {}

# Carrega cada base de dados em txt como data frame, a chave do dicionário é o nome do arquivo
for arquivo in lista_arquivos:
    dic_df[arquivo] = pd.read_csv(arquivo, sep=";")

In [276]:
# Atribui aos data frames mensais apenas os registros do município indicado
for df in dic_df:
    dic_df[df] = dic_df[df][dic_df[df]['município'] == codigo_municipio]

In [277]:
# Carrega a lista de Atividades Características de Turismo
df_act = pd.read_excel('Lista_ACT.xlsx', sheet_name='Lista ACT - Estrut OMT SUBCLASS')

# Armazena apenas a coluna com os códigos
df_act = df_act.drop(['Unnamed: 0', 'Unnamed: 2'], axis=1)

# Renomeia a coluna
df_act = df_act.rename(columns={df_act.columns[0]: "subclasse"})

# Armazena apenas os códigos com base no tamanho dos registros
df_act = df_act[df_act['subclasse'].str.len() == 10]

# Retira os símbolos do código, deixando apenas os algarismos
df_act.loc[:, 'subclasse'] = df_act['subclasse'].str.replace('.', '')
df_act.loc[:, 'subclasse'] = df_act['subclasse'].str.replace('-', '')
df_act.loc[:, 'subclasse'] = df_act['subclasse'].str.replace('/', '')

# Converte os registros para o tipo numérico (int)
df_act.loc[:, 'subclasse'] = pd.to_numeric(df_act['subclasse'])

In [278]:
# Cria dicionário para armazenar os empregos filtrados (apenas atividades turísticas)
empregos_turismo = {}

# Atribui ao dicionário apenas os dados do setor de turismo
for df in dic_df:
    empregos_turismo[df] = dic_df[df].merge(df_act, on='subclasse')
    
# Cria a coluna 'estoque' para armazenar a quantidade após cada movimentação
for mes in empregos_turismo:
    empregos_turismo[mes]['estoque'] = None

In [279]:
# Abre planilha do Ministério do Turismo com o estoque de empregos no turismo em Janeiro de 2020
df_estoque = pd.read_excel('Estoques do Turismo.xlsx', sheet_name='Estoque por Município')

# Ajusta os campos vindos do excel para facilitar consulta como data frame
# Altera o nome das colunas
for i in range(3, 59):
    df_estoque = df_estoque.rename({f'Unnamed: {i}':int(df_estoque.loc[1,f'Unnamed: {i}'])}, axis=1)
df_estoque = df_estoque.rename({'Tabela 2 - Estoques subclasse turismo em 1º de janeiro de 2020 por município':df_estoque.loc[0,'Tabela 2 - Estoques subclasse turismo em 1º de janeiro de 2020 por município']}, axis=1)
df_estoque = df_estoque.rename({'Unnamed: 1':df_estoque.loc[0,'Unnamed: 1']}, axis=1)
df_estoque = df_estoque.rename({'Unnamed: 2':df_estoque.loc[0,'Unnamed: 2']}, axis=1)
# Exclui as linhas sem registros
df_estoque = df_estoque.drop([0, 1, 5252,5253,5254], axis=0)
# Reseta o index
df_estoque = df_estoque.reset_index(drop=True)

# Atribui a soma de todos os estoque por CNAE a uma variável, estoque total dos CNAEs do Turismo
valor_estoque = df_estoque[df_estoque['Município cod'] == codigo_municipio].sum(axis=1)
valor_estoque = int(valor_estoque.values[0])

In [280]:
# Atribui o valor do estoque anterior o valor do estoque oriundo da planilha de estoques de 1/1/2020
estoque_anterior = valor_estoque

# Alimenta os data frames mensais com o valor do estoque a cada movimentação
for mes in lista_arquivos:
    # Primeiro estoque recebe o estoque anterior mais a primeira movimentação
    empregos_turismo[mes].loc[0, 'estoque'] = estoque_anterior + empregos_turismo[mes].loc[0, 'saldomovimentação']
    
    # Atualiza o estoque para cada movimentação
    for i in range(len(list(empregos_turismo[mes]['estoque']))):
        empregos_turismo[mes].loc[1:, 'estoque'] = empregos_turismo[mes]['estoque'].shift() + empregos_turismo[mes]['saldomovimentação']
    
    # Atribui o último estoque a variável de estoque anterior, para alimentar o primeiro registro do próximo data frame
    estoque_anterior = empregos_turismo[mes].iloc[-1,-1]

KeyError: 0

In [ ]:
# Cria lista com os dados finais de estoque de cada mês para criação do gráfico
plotagem = []

# Para cada data frame, atribui à lista de plotagem uma tupla com o 
for mes in empregos_turismo:
    plotagem.append([mes.replace('.txt', '').replace('CAGEDESTAB', '')[0:4], mes.replace('.txt', '').replace('CAGEDESTAB', '')[4:], empregos_turismo[mes].iloc[-1,-1]])

# Altera os meses de número para as 3 letras iniciais
for item in plotagem:
    if '01' in item[1]:
        item[1] = 'Jan'
    if '02' in item[1]:
        item[1] = 'Fev'
    if '03' in item[1]:
        item[1] = 'Mar'
    if '04' in item[1]:
        item[1] = 'Abr'
    if '05' in item[1]:
        item[1] = 'Mai'
    if '06' in item[1]:
        item[1] = 'Jun'
    if '07' in item[1]:
        item[1] = 'Jul'
    if '08' in item[1]:
        item[1] = 'Ago'
    if '09' in item[1]:
        item[1] = 'Set'
    if '10' in item[1]:
        item[1] = 'Out'
    if '11' in item[1]:
        item[1] = 'Nov'
    if '12' in item[1]:
        item[1] = 'Dez'

In [ ]:
# Atribui a lista de listas a um data frame
df_plotagem = pd.DataFrame(plotagem)

# Altera o nome das colunas
df_plotagem = df_plotagem.rename({0:'Ano',1:'Mês',2:'Empregados no Setor de Turismo'}, axis=1)

In [ ]:
# Cria o gráfico de linhas demonstrando a evolução dos empregos
fig = px.line(df_plotagem, x="Mês", y="Empregados no Setor de Turismo", text="Empregados no Setor de Turismo", color='Ano', title=f'Empregos no Turismo em {municipio}')
fig.update_traces(textposition="top right")
fig.show()